# Azure CLU 
- 라벨링 : CLU-EmailAppDemo

In [2]:
import requests
import uuid
import os
from dotenv import load_dotenv

load_dotenv()
OCP_API_SUBSCRIPTION_KEY = os.getenv("CLU_OCP_SUBSCRIPTION_KEY")
CLU_APIM_REQUEST_ID = os.getenv("CLU_APIM_REQUEST_ID")

def request_clu(text):
    endpoint = "https://ms6a005.cognitiveservices.azure.com/language/:analyze-conversations?api-version=2024-11-15-preview"

    headers ={
        "Ocp-Apim-Subscription-Key":OCP_API_SUBSCRIPTION_KEY,
        "Apim-Request-Id":CLU_APIM_REQUEST_ID,
        "Content-Type":"application/json"
    }
    body = {
    "kind": "Conversation",
    "analysisInput": {
        "conversationItem": {
            "id": "12345",
            "text": text,
            "modality": "text",
            "language": "en",
            "participantId": "user123"
        }
    },
    "parameters": {
        "projectName": "email",
        "verbose": True,
        "deploymentName": "EmailModel",
        "stringIndexType": "TextElement_V8"
    }
}

    response = requests.post(endpoint, headers=headers, json=body)
    if response.status_code == 200:
        response_json = response.json()
        top_intent = response_json['result']['prediction']['topIntent']
        intent_list = response_json['result']['prediction']['intents']
        entity_list = response_json['result']['prediction']['entities']

        return (top_intent, intent_list, entity_list)
    else:
        return None

# OpenAI GPT 4o-mini

In [3]:
import requests
import re
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GPT_API_KEY")

# Azure OpenAI API 엔드포인트 및 API 키
endpoint = "https://6a005-openai-eastus2.openai.azure.com/openai/deployments/6a005-gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"
api_key = API_KEY

# Azure Search 관련 설정
# ai_search_endpoint = "https://6a005-ai-search.search.windows.net"
# ai_search_api_key = ""
# ai_search_index = "venture-index-microsoft"

def request_gpt(prompt):
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    
    body = {
        "messages": [
            {
                "role": "system",
                "content": "너는 이메일 작업을 수행하는 AI Agent 야"
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }
    
    response = requests.post(endpoint, headers=headers, json=body)
    
    if response.status_code == 200:
        response_json = response.json()
        message = response_json['choices'][0]['message']
        # context가 없는 경우도 있을 수 있으므로 안전하게 가져옴
        citation_list = message.get('context', {}).get('citations', [])
        content = message['content']
        # [doc숫자]를 [참조 숫자]로 변환
        content = re.sub(r'\[doc(\d+)\]', r'[참조 \1]', content)
        return content, citation_list
    else:
        print("Error:", response.status_code, response.text)
        return "응답 에러 발생", []

# CLU의 intent, entity 처리 로직

In [4]:
def process_intent(top_intent, entity_list):
    
    if top_intent == "AddFlag":
        # 이메일에 표시 등 추가
        sender = None
        for ent in entity_list:
            if ent["category"] == "SenderName":
                sender = ent["text"]
                break
        if sender:
            return f"발신자 '{sender}'가 보낸 이메일에 플래그를 추가"
        else:
            return "이메일에 플래그를 추가"
    
    elif top_intent == "AddMore":
        # 파일을 이메일에 첨부
        attachments = []
        message_text = None
        for ent in entity_list:
            if ent["category"] == "Attachment":
                attachments.append(ent["text"])
            elif ent["category"] == "Message":
                message_text = ent["text"]
        
        if attachments or message_text:
            return f"이메일에 추가 내용을 첨부 (파일: {attachments}, 메시지: {message_text})"
        else:
            return "이메일에 추가할 내용을 첨부"
    
    elif top_intent == "Cancel":
        # 이전 작업 취소
        return "이전 작업을 취소"
    
    elif top_intent == "CheckMessages":
        # 새 이메일 확인
        return "현재 새 이메일이 없음"
    
    elif top_intent == "Confirm":
        # 사용자 확인
        return "작업을 승인함."
    
    elif top_intent == "Delete":
        # 이메일 삭제
        sender = None
        for ent in entity_list:
            if ent["category"] in ["SenderName", "ContactName", "FromRelationshipName"]:
                sender = ent["text"]
                break
        if sender:
            return f"'{sender}'가 보낸 이메일을 삭제"
        else:
            return "해당 이메일을 삭제"
    
    elif top_intent == "Forward":
        recipients = []
        for ent in entity_list:
            if ent["category"] in ["ContactName", "RelationshipName"]:
                recipients.append(ent["text"])
        if recipients:
            return f"이메일을 {', '.join(recipients)} 님께 전달"
        else:
            return "이메일을 전달"
    
    elif top_intent == "QueryLastText":
        # 마지막 메일
        return "가장 최근에 받은 이메일은 'OOO'님이 보냄 "
    
    elif top_intent == "ReadAloud":
        # 이메일을 음성으로
        return "아직 준비 중인 기능"
    
    elif top_intent == "Reply":
        # 답장 작성
        recipient = None
        for ent in entity_list:
            if ent["category"] == "ContactName":
                recipient = ent["text"]
        if recipient:
            return f"{recipient} 님께 답장을 작성"
        else:
            return "이메일에 답장을 작성"
    
    elif top_intent == "SearchMessages":
        # 키워드 검색
        search_texts = []
        for ent in entity_list:
            if ent["category"] in ["SearchTexts", "EmailSubject"]:
                search_texts.append(ent["text"])
        if search_texts:
            return f"'{', '.join(search_texts)}' 키워드로 이메일을 검색"
        else:
            return "이메일을 검색했습니다."
    
    elif top_intent == "SendEmail":
        # 새 이메일 전송
        return "새 이메일을 성공적으로 전송"
    
    elif top_intent == "ShowNext":
        # 다음 이메일 표시
        return "다음 이메일을 표시"
    
    elif top_intent == "ShowPrevious":
        # 이전 이메일/메시지 표시
        return "이전 이메일을 표시"
    
    else:  # top_intent == "None"
        return "명령을 이해하지 못함 (None 의도)"

# 동작 예시 함수
CLUE -> process_intent -> process_intent 결과를 GPT에게 프롬프트로 전달 -> GPT의 응답을 사용자에게 제공

In [5]:
def main_flow(user_input):
    # CLU
    clu_result = request_clu(user_input)

    top_intent, intent_list, entity_list = clu_result
    print("[CLU 결과]")
    print("Top Intent:", top_intent)
    print("Entities:", entity_list)
    print()
    
    # process_intent
    result = process_intent(top_intent, entity_list)
    print("[업무 결과]")
    print(result)
    print()

    # GPT에 전달할 프롬프트
    prompt_text = (
        f"다음은 이메일 작업 처리 결과야:\n"
        f"{result}\n\n"
        "이 내용을 사용자에게 친절하게 안내해주는 메시지를 작성해."
    )
    print('[prompt_text]')
    print(prompt_text)
    print()
    
    # GPT 호출
    gpt_response, citations = request_gpt(prompt_text)
    print("[GPT 최종 응답]")
    print(gpt_response)
    if citations:
        print("[참조 문서]", citations)

# 예시 실행
user_input = "any new email ?"
main_flow(user_input)


[CLU 결과]
Top Intent: CheckMessages
Entities: [{'category': 'Category', 'text': 'new', 'offset': 4, 'length': 3, 'confidenceScore': 1}]

[업무 결과]
현재 새 이메일이 없음

[prompt_text]
다음은 이메일 작업 처리 결과야:
현재 새 이메일이 없음

이 내용을 사용자에게 친절하게 안내해주는 메시지를 작성해.

[GPT 최종 응답]
안녕하세요! 현재 새 이메일이 없습니다. 필요하신 사항이나 문의가 있으시면 언제든지 말씀해 주세요. 도움이 필요하시면 기꺼이 도와드리겠습니다! 감사합니다.


# Gradio

In [ ]:
import gradio as gr

def main_flow_string(user_input):

    # CLU -> 의도 처리 -> GPT 로직 -> GPT 응답(gpt_response)만 반환

    # CLU 호출
    clu_result = request_clu(user_input)
    if clu_result is None:
        return "오류: CLU 호출 실패"

    top_intent, intent_list, entity_list = clu_result

    # 의도 처리
    summary = process_intent(top_intent, entity_list)

    # GPT 프롬프트
    prompt_text = (
        f"다음은 이메일 작업 처리 결과입니다:\n"
        f"{summary}\n\n"
        "이 내용을 사용자에게 친절하게 안내해주는 짧은 메시지를 작성해 주세요."
    )

    # GPT 호출
    gpt_response, citations = request_gpt(prompt_text)

    # GPT 응답만 리턴
    return gpt_response

def click_send(prompt, history):
    """
    Gradio에서 '전송' 버튼을 눌렀을 때 호출되는 함수.
    - prompt: 사용자가 입력한 텍스트
    - history: 현재까지의 대화 기록 ([(role, message), (role, message), ...] 형태)
    """
    # 1) main_flow_string() 호출 → CLU + GPT 결과 문자열 얻기
    full_result = main_flow_string(prompt)

    # 2) 대화 기록에 사용자 메시지 & 결과 문자열 추가
    history.append(("사용자", prompt))
    history.append(("GPT", full_result))

    # 3) (새로운 대화 기록, 참조 문서, 입력창 초기화)
    # 여기서는 main_flow_string() 내부에 citations가 이미 문자열로 포함되므로,
    # 따로 참조 문서를 파싱하지 않고, 빈 문자열을 반환합니다.
    return history, "", ""

# Gradio 앱 구성
with gr.Blocks() as demo:
    gr.Markdown("## Email AI Agent")

    chatbot = gr.Chatbot(label="Agent", height=300)
    prompt = gr.Textbox(label="지시 사항", placeholder="이메일 지시 사항을 입력하세요.")
    send_btn = gr.Button("전송")

    # 참조 텍스트 박스 (필요하다면 사용)
    references_box = gr.Textbox(label="참조", lines=3)

    # 전송 버튼 클릭 → click_send 함수 호출
    send_btn.click(
        fn=click_send,
        inputs=[prompt, chatbot],
        outputs=[chatbot, references_box, prompt]
    )

demo.launch()